# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import logging
import os
import csv
import joblib 

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from azureml.data.datapath import DataPath
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.data.dataset_factory import TabularDatasetFactory
from sklearn.model_selection import train_test_split
from azureml.pipeline.steps import AutoMLStep
from azureml.widgets import RunDetails

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.20.0


In [2]:
#import pandas as pd
#import numpy as np

#https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/automated-machine-learning/classification-bank-marketing-all-features/auto-ml-classification-bank-marketing-all-features.ipynb

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.

In the Automated ML run a classification task ............ is done using the .........dataset.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [3]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

# choose a name for experiment
experiment_name = 'capstone-automl-exp'
project_folder = './pipeline-project'  #kann vielleicht entfallen

experiment=Experiment(ws, experiment_name)
experiment

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code AKP2KTPWL to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.
quick-starts-ws-139398
aml-quickstarts-139398
southcentralus
3e42d11f-d64d-4173-af9b-12ecaa1030b3


Name,Workspace,Report Page,Docs Page
capstone-automl-exp,quick-starts-ws-139398,Link to Azure Machine Learning studio,Link to Documentation


Create compute cluster

In [4]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

amlcompute_cluster_name = "cluster-automl"

try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4,
                                                           min_nodes=1)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 5) #timeout_in_minutes = 10

Creating
Succeeded.......................
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


Embed the dataset

In [5]:
found = False
key = "heart-db" #rename->chose dataset
description_text = "Kaggle heard data database" #rename

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 
        #print("found!")

if not found: 
        # Create AML Dataset and register it into Workspace
        #print("not found!") #
        example_data = 'https://raw.githubusercontent.com/Daniel-car1/nd00333-capstone/main/heart.csv'  
        dataset = Dataset.Tabular.from_delimited_files(example_data, header=True)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = dataset.to_pandas_dataframe()
df.describe()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
count,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000
mean,54.366337,0.683168,0.966997,131.623762,246.264026,0.148515,0.528053,149.646865,0.326733,1.039604,1.399340,0.729373,2.313531,0.544554
std,9.082101,0.466011,1.032052,17.538143,51.830751,0.356198,0.525860,22.905161,0.469794,1.161075,0.616226,1.022606,0.612277,0.498835
min,29.000000,0.000000,0.000000,94.000000,126.000000,0.000000,0.000000,71.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,47.500000,0.000000,0.000000,120.000000,211.000000,0.000000,0.000000,133.500000,0.000000,0.000000,1.000000,0.000000,2.000000,0.000000
50%,55.000000,1.000000,1.000000,130.000000,240.000000,0.000000,1.000000,153.000000,0.000000,0.800000,1.000000,0.000000,2.000000,1.000000
75%,61.000000,1.000000,2.000000,140.000000,274.500000,0.000000,1.000000,166.000000,1.000000,1.600000,2.000000,1.000000,3.000000,1.000000
max,77.000000,1.000000,3.000000,200.000000,564.000000,1.000000,2.000000,202.000000,1.000000,6.200000,2.000000,4.000000,3.000000,1.000000


In [6]:
dataset.take(5).to_pandas_dataframe()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [7]:
train, test = train_test_split(df, test_size=0.2, random_state=1 )

In [8]:
if "training" not in os.listdir():
    os.mkdir("./training")

if "models" not in os.listdir():
    os.mkdir("./models")
if "outputs" not in os.listdir():
    os.mkdir("./outputs")

train.to_csv("training/train_data.csv", index = False)

In [9]:
data_store = ws.get_default_datastore()
data_store.upload(src_dir = "./training", target_path = 'udacity-project', overwrite = True,  show_progress = True)

Uploading an estimated of 1 files
Uploading ./training/train_data.csv
Uploaded ./training/train_data.csv, 1 files out of an estimated total of 1
Uploaded 1 files


$AZUREML_DATAREFERENCE_0664bb510e3f4f058fa263e3cb08bcea

In [10]:
train_data = Dataset.Tabular.from_delimited_files(path=data_store.path("udacity-project/train_data.csv"))

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [63]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 20,
    "n_cross_validations": 3,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'AUC_weighted'
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="target",    
                             path = project_folder,
                             enable_onnx_compatible_models=True,
                             enable_early_stopping= True,
                             featurization= 'off', #auto
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

In [64]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output= True)

Running on remote.
No run_configuration provided, running on cluster-automl with default configuration
Running on remote compute: cluster-automl
Parent Run ID: AutoML_b7a4272f-660c-4072-a222-af6fe4a1878d

Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

****************************************************************************************************
ITERATION: The iteration being evaluated.
PIPELINE: A summary description of the pipeline being evaluated.
DURATION: Time taken for the current iteration.
METRIC: The r

In [65]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…



****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

****************************************************************************************************
ITERATION: The iteration being evaluated.
PIPELINE: A summary description of the pipeline being evaluated.
DURATION: Time taken for the current iteration.
METRIC: The result of computing score on the fitted pipeline.
BEST: The best observed score thus far.
****************************************************************************************************

 ITERATION   PIPELINE                                       DURATION    

{'runId': 'AutoML_b7a4272f-660c-4072-a222-af6fe4a1878d',
 'target': 'cluster-automl',
 'status': 'Completed',
 'startTimeUtc': '2021-02-23T20:59:09.547926Z',
 'endTimeUtc': '2021-02-23T21:32:35.242139Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'AUC_weighted',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '3',
  'target': 'cluster-automl',
  'AMLSettingsJsonString': '{"path":null,"name":"capstone-automl-exp","subscription_id":"3e42d11f-d64d-4173-af9b-12ecaa1030b3","resource_group":"aml-quickstarts-139398","workspace_name":"quick-starts-ws-139398","region":"southcentralus","compute_target":"cluster-automl","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"AUC_weighted","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":3,"y_min":null,"y_max":nul

In [66]:
best_model, fitted_model = remote_run.get_output()

Package:azureml-automl-runtime, training version:1.22.0, current version:1.20.0
Package:azureml-core, training version:1.22.0, current version:1.20.0
Package:azureml-dataprep, training version:2.9.1, current version:2.7.3
Package:azureml-dataprep-native, training version:29.0.0, current version:27.0.0
Package:azureml-dataprep-rslex, training version:1.7.0, current version:1.5.0
Package:azureml-dataset-runtime, training version:1.22.0, current version:1.20.0
Package:azureml-defaults, training version:1.22.0, current version:1.20.0
Package:azureml-interpret, training version:1.22.0, current version:1.20.0
Package:azureml-pipeline-core, training version:1.22.0, current version:1.20.0
Package:azureml-telemetry, training version:1.22.0, current version:1.20.0
Package:azureml-train-automl-client, training version:1.22.0, current version:1.20.0
Package:azureml-train-automl-runtime, training version:1.22.0, current version:1.20.0


In [67]:
print(best_model)

Run(Experiment: capstone-automl-exp,
Id: AutoML_b7a4272f-660c-4072-a222-af6fe4a1878d_41,
Type: azureml.scriptrun,
Status: Completed)


In [68]:
print(fitted_model)

Pipeline(memory=None,
         steps=[('prefittedsoftvotingclassifier',
                 PreFittedSoftVotingClassifier(classification_labels=None,
                                               estimators=[('22',
                                                            Pipeline(memory=None,
                                                                     steps=[('MaxAbsScaler',
                                                                             MaxAbsScaler(copy=True)),
                                                                            ('ExtraTreesClassifier',
                                                                             ExtraTreesClassifier(bootstrap=True,
                                                                                                  ccp_alpha=0.0,
                                                                                                  class_weight='balanced',
                                                          

In [69]:
best_model.download_file('outputs/model.pkl', 'outputs/automl_hearth.pkl')
best_model.download_file('outputs/scoring_file_v_1_0_0.py', 'outputs/score_aml.py')
best_model.download_file('automl_driver.py', 'outputs/automl_driver.py')


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [72]:


model = best_model.register_model(model_name='best_automl_model', 
                                        model_path='outputs/model.pkl',
                                        tags= {'Training context':'Auto ML'}#,
                                        #properties={
                                        #     'AUC_weighted':
                                        #     get_best_automl_metrics['AUC_weighted']
                                        # }
                                        # description="Training context : AutoML")
                                        )
print(model)

Model(workspace=Workspace.create(name='quick-starts-ws-139398', subscription_id='3e42d11f-d64d-4173-af9b-12ecaa1030b3', resource_group='aml-quickstarts-139398'), name=best_automl_model, id=best_automl_model:1, version=1, tags={'Training context': 'Auto ML'}, properties={})


In [73]:
import onnxruntime

In [74]:
best_run, onnx_mdl = remote_run.get_output(return_onnx_model=True)

In [75]:
from azureml.automl.runtime.onnx_convert import OnnxConverter
onnx_fl_path = "./best_model.onnx"
OnnxConverter.save_onnx_model(onnx_mdl, onnx_fl_path)


In [77]:
model_name = best_run.properties['model_name']
print(best_run)

Run(Experiment: capstone-automl-exp,
Id: AutoML_b7a4272f-660c-4072-a222-af6fe4a1878d_41,
Type: azureml.scriptrun,
Status: Completed)


In [78]:
print(fitted_model[-1])

PreFittedSoftVotingClassifier(classification_labels=None,
                              estimators=[('22',
                                           Pipeline(memory=None,
                                                    steps=[('MaxAbsScaler',
                                                            MaxAbsScaler(copy=True)),
                                                           ('ExtraTreesClassifier',
                                                            ExtraTreesClassifier(bootstrap=True,
                                                                                 ccp_alpha=0.0,
                                                                                 class_weight='balanced',
                                                                                 criterion='gini',
                                                                                 max_depth=None,
                                                                                 max_fe

In [79]:
best_run.get_metrics()

{'AUC_macro': 0.9245709187969441,
 'log_loss': 0.45872283236951455,
 'recall_score_macro': 0.8420016353719714,
 'precision_score_weighted': 0.8505326860569458,
 'f1_score_weighted': 0.8469276841884055,
 'AUC_weighted': 0.9245709187969441,
 'f1_score_macro': 0.8447423235148249,
 'precision_score_micro': 0.8481848184818482,
 'AUC_micro': 0.9227526713067347,
 'norm_macro_recall': 0.6840032707439428,
 'recall_score_weighted': 0.8481848184818482,
 'average_precision_score_macro': 0.926570537683543,
 'precision_score_macro': 0.8524634683497982,
 'recall_score_micro': 0.8481848184818482,
 'weighted_accuracy': 0.8542554988104948,
 'matthews_correlation': 0.6943820577562994,
 'average_precision_score_weighted': 0.9273000554605774,
 'accuracy': 0.8481848184818482,
 'f1_score_micro': 0.8481848184818482,
 'balanced_accuracy': 0.8420016353719714,
 'average_precision_score_micro': 0.9222358786022634,
 'confusion_matrix': 'aml://artifactId/ExperimentRun/dcid.AutoML_b7a4272f-660c-4072-a222-af6fe4a1878

In [80]:
script_file_name = 'inference/score.py'

best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'inference/score.py')

In [81]:
#TODO: Save the best model

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [82]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import LocalWebservice, AciWebservice
from azureml.core.webservice import Webservice
from azureml.core.model import Model
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.automl.core.shared import constants

In [83]:
#Deploy model and check service state
deployment_config = AciWebservice.deploy_configuration(
    cpu_cores=1,
    memory_gb=1,
    description='Heart failure prediction', #vielleicht den Text ändern
    auth_enabled=True
)

In [84]:
model = Model(workspace = ws,
                       model_path='outputs/model.pkl',
                       name='best_automl_model')

In [85]:
myenv = best_model.get_environment()
best_model.download_file('outputs/scoring_file_v_1_0_0.py', 'score.py')
best_model.download_file(constants.CONDA_ENV_FILE_PATH, 'myenv.yml')

inference_config = InferenceConfig(entry_script='score.py', environment=myenv)

service = Model.deploy(workspace=ws, 
                       name='aciservice', 
                       models=[model], 
                       inference_config=inference_config, 
                       deployment_config=deployment_config)
        
# model = Model.register(model_path = "./models/automl_hearth.pkl",
#                       model_name = "automl_hearth.pkl",
#                       description = "Best model trained with AutoML",
#                       workspace = ws, model_framework=Model.Framework.SCIKITLEARN)
service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running....................................
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [86]:
#Enable application insights
service.update(enable_app_insights = True)

In [87]:
print("State : "+service.state)
print("Key " + service.get_keys()[0])
print("Swagger URI : "+service.swagger_uri)
print("Scoring URI : "+service.scoring_uri)

State : Healthy
Key QFUB53kwpCXQTkFSAjUx96wtdeqaRNjS
Swagger URI : http://5fd5cea9-e482-4282-b8c7-975a94ccfd91.southcentralus.azurecontainer.io/swagger.json
Scoring URI : http://5fd5cea9-e482-4282-b8c7-975a94ccfd91.southcentralus.azurecontainer.io/score


In [92]:
print(service.get_logs())

2021-02-23T21:41:56,114545400+00:00 - iot-server/run 
2021-02-23T21:41:56,117673500+00:00 - nginx/run 
2021-02-23T21:41:56,118597500+00:00 - rsyslog/run 
2021-02-23T21:41:56,122369100+00:00 - gunicorn/run 
/usr/sbin/nginx: /azureml-envs/azureml_09ff55f546b313bb1ab136a466214499/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_09ff55f546b313bb1ab136a466214499/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_09ff55f546b313bb1ab136a466214499/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_09ff55f546b313bb1ab136a466214499/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_09ff55f546b313bb1ab136a466214499/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
rsyslogd

In [93]:
#Deploy model and check service state
print(service.state)

Healthy


In [94]:
print(service.scoring_uri)

http://5fd5cea9-e482-4282-b8c7-975a94ccfd91.southcentralus.azurecontainer.io/score


In [115]:
import json
import requests

scoring_uri = 'http://5fd5cea9-e482-4282-b8c7-975a94ccfd91.southcentralus.azurecontainer.io/score' 
#key=''
headers = {'Content-Type': 'application/json'}
# If authentication is enabled, set the authorization header
#headers['Authorization'] = f'Bearer {key}'
headers['Authorization'] = f'Bearer {service.get_keys()[0]}'

data = {"data":
        [
          {
            "age":37,
            "sex": 1,
            "cp": 2,
            "trestbps": 130,
            "chol": 250,
            "fbs": 0,
            "restecg": 1,
            "thalach": 187,
            "exang": 0,
            "oldpeak": 3.5,
            "slope": 0,
            "ca": 0,
            "thal": 2,
          }
      ]
    }
test_sample = json.dumps(data)


resp = requests.post(scoring_uri, test_sample, headers=headers)

In [116]:
myenv.save_to_directory('./training',overwrite=True)

In [117]:
print(resp)

<Response [200]>


In [118]:
print(resp.status_code)
print(resp.elapsed)
print(resp.json())

200
0:00:00.211210
{"result": [1]}


In [119]:
print('expected result: ', df['target'][1])

expected result:  1


In [120]:
print(service.get_logs())

2021-02-23T21:41:56,114545400+00:00 - iot-server/run 
2021-02-23T21:41:56,117673500+00:00 - nginx/run 
2021-02-23T21:41:56,118597500+00:00 - rsyslog/run 
2021-02-23T21:41:56,122369100+00:00 - gunicorn/run 
/usr/sbin/nginx: /azureml-envs/azureml_09ff55f546b313bb1ab136a466214499/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_09ff55f546b313bb1ab136a466214499/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_09ff55f546b313bb1ab136a466214499/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_09ff55f546b313bb1ab136a466214499/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_09ff55f546b313bb1ab136a466214499/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
rsyslogd

In [ ]:
#service.delete()

In [ ]:
#compute_target.delete()

In [ ]:
print(finished)

In [ ]:
#conda activate myenv

In [ ]:
#conda env export > env.yml

In [ ]:
#from azureml.automl.core.shared import constants
#best_run.download_file(constants.CONDA_ENV_FILE_PATH, 'myenv.yml')

In [ ]:
#from azureml.core import Environment
#env = Environment.from_conda_specification(name='myenv',file_path = 'env.yml')



#inference_config = InferenceConfig(entry_script='./score.py',
#                                    environment=env)

#f = open("envnew.yml", "w")
#f.write(env.python.conda_dependencies.serialize_to_string())
#f.close()

#print("packages", env.python.conda_dependencies.serialize_to_string())

In [ ]:
#service_name = 'heart_service'

#service = Model.deploy(ws, service_name, [model], inference_config, deployment_config=aci_config)

In [ ]:
#service.wait_for_deployment(True)
#print(service.state)

In [ ]:
#print("scoring URI: " + service.scoring_uri)

print("Result:",response.text)

print(service.get_logs())

print(response.status_code)
print(response.elapsed)
print(response.json())

service.delete()

compute_target.delete()

In [ ]:
#Ende

In [ ]:
# The following cells were changed from code -> markdown

#Register the Fitted Model for Deployment
model_name = best_run.properties['model_name']
description = 'AutoML Model trained on heart data'
tags = None
model = remote_run.register_model(model_name = model_name, description = description, tags = tags)

print(remote_run.model_id) # This will be written to the script file later in the notebook.  

from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core.webservice import Webservice
from azureml.core.model import Model
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies

myenv = CondaDependencies.create(pip_packages=["numpy","onnxruntime","azureml-core","azrueml-defaults"])
with open("myenv.yml","w") as f:
    f.write(myenv.serialize_to_string())



myenv = Environment.from_conda_specification(name="myenv", file_path="myenv.yml")
inference_config = InferenceConfig(entry_script=script_file_name, environment=myenv)

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                               memory_gb = 2, 
                                               enable_app_insights=True, 
                                               auth_enabled=True,
                                               tags = {'area': "haData", 'type': "automl_classification"},
                                               description = 'service for Automl Classification')



aci_service_name = 'automlheartendpoint2'
print(aci_service_name)
aci_service = Model.deploy(ws, aci_service_name, [model], inference_config, aciconfig) #,overwrite=True
aci_service.wait_for_deployment(show_output=True)
print(aci_service.state)

#logging
from azureml.core import Workspace
from azureml.core.webservice import Webservice
#ws = Workspace.from_config()
service = Webservice(name=aci_service_name, workspace=ws)
logs = service.get_logs()

for line in logs.split('\n'):
    print(line)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
#den absatz vielleicht wegwerfen?!

#import json
#import requests

#X_test_json = X_test.to_json(orient='records')
#data = "{\"data\": " + X_test_json +"}"
#headers = {'Content-Type': 'application/json'}

#resp = requests.post(aci_service.scoring_uri, data, headers=headers)

#y_pred = json.loads(json.loads(resp.text))['result']

import json
import requests

scoring_uri = aci_service.scoring_uri() 
#key=''
headers = {'Content-Type': 'application/json'}
# If authentication is enabled, set the authorization header
#headers['Authorization'] = f'Bearer {key}'
data = {"data":
        [
          {
            "age":37,
            "sex": 1,
            "cp": 2,
            "trestbps": 130,
            "chol": 250,
            "fbs": 0,
            "restecg": 1,
            "thalach": 187,
            "exang": 0,
            "oldpeak": 3.5,
            "slope": 0,
            "ca": 0,
            "thal": 2,
          }
      ]
    }
test_sample = json.dumps(data)


resp = requests.post(scoring_uri, test_sample, headers=headers)


print("prediction ", resp.json())

y_pred = json.loads(json.loads(resp.text))['result']
print("prediction ", y_pred)

print("expected result: 1")

TODO: In the cell below, print the logs of the web service and delete the service

logs = service.get_logs()
for line in logs.split('\n'):
    print(line)

service.delete()

compute_target.delete()

print('finished')